In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
df = pd.read_csv('data_cleaned.csv')


In [36]:
df['İlçe'] = df['İlçe'].str.strip()
df['Mahalle'] = df['Mahalle'].str.strip()
df['İlçe'] = df['İlçe'].astype('category')
df['Mahalle'] = df['Mahalle'].astype('category')
df['Oda Sayısı'] = df['Oda Sayısı'].astype('int')
df['m^2'] = df['m^2'].astype('int')
df['Bina Yaşı'] = df['Bina Yaşı'].astype('int')
df['Kat'] = df['Kat'].astype('int')
df['Fiyat'] = df['Fiyat'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8512 entries, 0 to 8511
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   İlçe        8512 non-null   category
 1   Mahalle     8512 non-null   category
 2   Oda Sayısı  8512 non-null   int64   
 3   m^2         8512 non-null   int64   
 4   Bina Yaşı   8512 non-null   int64   
 5   Kat         8512 non-null   int64   
 6   Fiyat       8512 non-null   int64   
dtypes: category(2), int64(5)
memory usage: 369.3 KB


In [37]:
categorical_features = ['İlçe', 'Mahalle']
numerical_features = ['Oda Sayısı', 'm^2','Bina Yaşı', 'Kat']

In [38]:
#veri ön işleme pipeline'ı oluşuturuluyor. amaç hem sayısal hem de kategorik değişkenlere farklı dönüşün işlemleri uygulamak
#column transformer'ın amacı farklı sütun türlerine (nümerik ve kategorik), birden fazla dönüşüm uygulamak için
#sayısal değerleri 1 ve 0 arasında standardize ediyoruz. 
#kategorik sütunlara one hot encoding uygulanır, handle unknown ise eğitim sırasında karşılaşılmayan veri testte karşımıza çıkarsa hata almamak içind
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [39]:
#makine öğrenimi algoritması olarak kullanacağımız lineer regresyon'da bağımlı ve bağımsız değişkenleri atadık
X = df.drop('Fiyat', axis=1)
y = df['Fiyat']


In [40]:
bins = [x for x in range(0, 70000, 10000)]
labels = [x for x in range(1, 7)]
print(bins)
print(labels)

[0, 10000, 20000, 30000, 40000, 50000, 60000]
[1, 2, 3, 4, 5, 6]


In [41]:
y = pd.cut(y, bins=bins, labels=labels)

In [42]:
print(y.unique())


[3, 4, 2, 5, 1]
Categories (6, int64): [1 < 2 < 3 < 4 < 5 < 6]


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
#model random forest sınıflandırmasına göre düzenlendi. 
model = Pipeline([
    ('preparation', full_pipeline),
    ('model', RandomForestClassifier(n_estimators=100))
])

In [45]:
model.fit(X_train, y_train)


Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Oda Sayısı', 'm^2',
                                                   'Bina Yaşı', 'Kat']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['İlçe', 'Mahalle'])])),
                ('model', RandomForestClassifier())])

In [46]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))


[[ 22  36   2   1   0]
 [  7 739 116   5   0]
 [  1 156 382  48   0]
 [  0   8  79  73   4]
 [  0   0  10   8   6]]


In [47]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.73      0.36      0.48        61
           2       0.79      0.85      0.82       867
           3       0.65      0.65      0.65       587
           4       0.54      0.45      0.49       164
           5       0.60      0.25      0.35        24

    accuracy                           0.72      1703
   macro avg       0.66      0.51      0.56      1703
weighted avg       0.71      0.72      0.71      1703



In [48]:
new_data = pd.DataFrame({
    'İlçe': ['Gölbaşı'],
    'Mahalle': ['Bahçelievler'],
    'Oda Sayısı': [3],
    'm^2': [90],
    'Bina Yaşı': [10],
    'Kat': [-1]
})

# Modelin tahmini
predicted_class = model.predict(new_data)
print("Tahmin Edilen Sınıf:", predicted_class)

Tahmin Edilen Sınıf: [2]
